In [16]:
import os,time,itertools,imageio,pickle
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [17]:
def lrelu(x,th=0.2):
    return tf.maximum(th*x,x)

In [18]:
def generator(x,isTrain=True,reuse=False):
    with tf.variable_scope('generator',reuse=reuse):
        
        conv1=tf.layers.conv2d_transpose(x,1024,[4,4],strides=(2,2),padding='valid')
        lrelu1=lrelu(tf.layers.batch_normalization(conv1,training=isTrain),0.2)
        
        conv2=tf.layers.conv2d_transpose(lrelu1,512,[4,4],strides=(2,2),padding='same')
        lrelu2=lrelu(tf.layers.batch_normalization(conv2,training=isTrain),0.2)
        
        conv3=tf.layers.conv2d_transpose(lrelu2(256,[4,4],strides=(2,2),padding='same'))
        lrelu3=lrelu(tf.layers.batch_normalization(conv3,training=isTrain),0.2)
        
        conv4=tf.layers.conv2d_transpose(lrelu3,128,[4,4],strides=(2,2),padding='same')
        lrelu4=lrelu(tf.layers.batch_normalization(conv4,training=isTrain),0.2)
        
        conv5=tf.layers.conv2d_transpose(lrelu,1,[4,4],strides=(2,2),padding='same')
        o=tf.nn.tanh(conv5)
        
        return o 
        

In [19]:
def discriminator(x,isTrai=True,reuse=False):
    with tf.variable_scope('discriminator',reuse=reuse):
        
        conv1=tf.layers.conv2d(x,128,[4,4],strides=(2,2),padding='same')
        lrelu1=lrelu(conv1,0.2)
        
        conv2=tf.layers.conv2d(lrelu1,256,[4,4],strides=(2,2),padding='same')
        lrelu2=lrelu(tf.layers.batch_normalization(conv2,training=isTrain),0.2)
        
        conv3=tf.layers.conv2d(lrelu1,512,[4,4],strides=(2,2),padding='same')
        lrelu3=lrelu(tf.layers.batch_normalization(conv3,training=isTrain),0.2)
        
        conv4=tf.layers.conv2d(lrelu1,1024,[4,4],strides=(2,2),padding='same')
        lrelu4=lrelu(tf.layers.batch_normalization(conv4,training=isTrain),0.2)
        
        conv5=tf.layers.conv2d(lrelu4,1,[4,4],strides=(1,1),padding='valid')
        o=tf.nn.sigmoid(conv5)
        
        return o,conv5
        

In [20]:
fixed_z=np.random.normal(0,1,(25,1,1,100))

def show_result(num_epoch,show=False,path='result.png'):
    test_images=sess.run(G_z,{z:fixed_z,isTrain:False})
    
    size_figure_grid=5
    
    fig,ax=plt.subplots(size_figure_grid,size_figure_grid,figsize=(5,5))
    
    for i,j in itertools.product(range(size_figure_grid),range(size_figure_grid)):
        ax[i,j].get_xaxis().set_visible(False)
        ax[i,j].get_yaxis().set_visible(False)
    
    for k in range(size_figure_grid*size_figure_grid):
        i=k // size_figure_grid
        j=k % size_figure_grid
        ax[i,j].cla()
        ax[i,j].imshow(np.reshape(test_images[k],(64,64)),cmap='gray')
        
        label='Epoch {0}'.format(num_epoch)
        fig.text(0.5,0.04,label,ha='center')
        
        if save:
            plt.savefig(path)
            
        if show:
            plt.show()
        else:
            plt.close()
            
            

In [21]:
def show_train_hist(hist,show=False,save=False,path='Train_hist.png'):
    x=range(len(hist['D_losses']))
    
    y1=hist['D_losses']
    y2=hist['G_losses']
    
    plt.plot(x,y1,label='D_loss')
    plt.plot(x,y2,label='G_loss')
    
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    
    plt.legend(loc=4)
    plt.grid(True)
    plt.tight_layout()
    
    if save:
        plt.savefig(path)
        
    if show:
        plt.show()
    
    else:
        plt.close()
        
        
        

In [22]:
batch_size=100
lr=0.0002
train_epoch=20

In [23]:
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True,reshape=[])

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [28]:
x=tf.placeholder(tf.float32,shape=(None,64,64,1))
z=tf.placeholder(tf.float32,shape=(None,1,1,100))
isTrain=tf.placeholder(dtype=tf.bool)



In [25]:
G_z=generator(z,isTrain)

D_real,D_real_logits=discriminator(x,isTrain)
D_fake,D_fake_logits=discriminator(G_z,isTrain,reuse=True)

ValueError: Variable generator/conv2d_transpose/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-6-983243963375>", line 4, in generator
    conv1=tf.layers.conv2d_transpose(x,1024,[4,4],strides=(2,2),padding='valid')
  File "<ipython-input-14-0201acd2462c>", line 1, in <module>
    G_z=generator(z,isTrain)
  File "/home/akinyalcinkaya/anaconda2/envs/tensorlow/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
